In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit import Aer
from qiskit_machine_learning.algorithms import QSVC
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_openml
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.visualization import plot_circuit_layout
from qiskit import QuantumCircuit
import numpy as np
from qiskit.circuit.library import BlueprintCircuit
from qiskit.circuit import Parameter
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

import qiskit
from qoopmaster.core import state
from qoopmaster.compilation.qsp import QuantumStatePreparation
from qoopmaster.evolution.environment_synthesis import MetadataSynthesis
from qoopmaster.evolution.generator import by_num_cnot # By num cnot
from qoopmaster.evolution.generator import by_num_cnot_old # By num cnot
from qoopmaster.evolution.generator import by_depth_new # By depth new

##################################################################


**Random point dataset**

In [2]:

from sklearn.datasets import make_blobs

def generate_data(n_samples,n_features,centers, random_state):
    X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)
    y = 2 * y - 1  # Convert labels to {-1, +1} for compatibility with QSVC
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train, y_test

Xr_train,Xr_test,yr_train, yr_test = generate_data(n_samples=120, n_features=4, centers=2, random_state=41)\
    
    
def train_qsvm_with_random(quantum_circuit):
    
    print(quantum_circuit.draw())
    print(f"Parameter: {len(quantum_circuit.parameters)}")
    backend = Aer.get_backend('statevector_simulator')
    quantum_kernel = QuantumKernel(feature_map=quantum_circuit, quantum_instance=backend)
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    qsvc.fit(Xr_train, yr_train)
    y_pred = qsvc.predict(Xr_test)
    accuracy = accuracy_score(yr_test, y_pred)

    return accuracy

In [3]:
Xr_train.shape

(96, 4)

**Wine dataset**

In [16]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

# Load Wine Dataset
wine = load_wine()
X = wine.data
y = wine.target

# Lấy dữ liệu chỉ gồm lớp 0 và 1
X = X[y != 2]
y = y[y != 2]

# Chia train-test
Xw_train, Xw_test, yw_train, yw_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
Xw_train = scaler.fit_transform(Xw_train)
Xw_test = scaler.transform(Xw_test)

# Áp dụng PCA
pca = PCA(n_components=4)
Xw_train = pca.fit_transform(Xw_train)
Xw_test = pca.transform(Xw_test)


# Thiết lập tùy chọn hiển thị của NumPy
np.set_printoptions(suppress=True)  # precision=3: 3 chữ số thập phân, suppress=True: bỏ ký hiệu khoa học


def train_qsvm_with_wine(quantum_circuit):
    print('parameter: ', len(quantum_circuit.parameters))
    backend = Aer.get_backend('statevector_simulator')
    quantum_kernel = QuantumKernel(feature_map=quantum_circuit, quantum_instance=backend)
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    qsvc.fit(Xw_train, yw_train)
    y_pred = qsvc.predict(Xw_test)
    accuracy = accuracy_score(yw_test, y_pred)

    return accuracy


print(f"{Xw_train.shape}")



(104, 4)


In [5]:
print(Xw_train)

[[-2.96566571 -1.19981356  0.200953    1.44581844]
 [-0.36333653 -1.36189673 -0.34597488  0.4696873 ]
 [ 2.59522139 -0.09800267 -1.6010798  -0.00284246]
 [ 3.22951382  0.06886644 -0.02558222  0.18915916]
 [ 1.63361267 -1.22613815 -0.04919233  0.0370117 ]
 [ 3.00563107  0.42960706 -0.941301    0.09456229]
 [ 2.39868742 -0.52568567  0.33147905 -0.8471472 ]
 [ 0.82212247  0.65259029  0.43176006 -1.02740405]
 [-1.36772313 -0.81344516 -1.1237484  -1.67591427]
 [-2.30968101 -0.39642454  1.04017436 -1.03724131]
 [-2.67735392  0.41222813  0.22542442 -1.4537568 ]
 [ 2.54055117 -1.34812208  1.62075776  0.27437308]
 [-2.82262766 -0.10534998  0.10388188  1.50770234]
 [ 1.62737727  0.08839331 -0.38437798 -0.76603469]
 [ 1.46730624  0.31642503 -1.27306447  0.09405107]
 [-0.23545791 -0.72467594  0.99286515 -0.48386237]
 [-1.71915134  0.48023949  0.94233759  0.68378904]
 [ 2.3257985  -0.00544558  1.49327704 -0.48153172]
 [-1.40461473 -2.40707683 -0.22717818 -0.39611149]
 [-0.03117362  4.2419963  -2.30

**Iris dataset**

In [6]:
from sklearn.datasets import load_iris
scaler = StandardScaler()
iris = load_iris()
Xi_train, Xi_test, yi_train, yi_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

def train_qsvm_with_Iris(quantum_circuit):
    
    print(quantum_circuit.draw())
    print(f"Parameter: {len(quantum_circuit.parameters)}")
    backend = Aer.get_backend('statevector_simulator')
    quantum_kernel = QuantumKernel(feature_map=quantum_circuit, quantum_instance=backend)
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    qsvc.fit(Xi_train, yi_train)
    y_pred = qsvc.predict(Xi_test)
    accuracy = accuracy_score(yi_test, y_pred)

    return accuracy


In [7]:
Xi_train.dtype

dtype('float64')

**Credit dataset**

In [8]:
from TestGAGermanCreditData import X_train as Xc_train
from TestGAGermanCreditData import X_test as Xc_test
from TestGAGermanCreditData import y_train as yc_train
from TestGAGermanCreditData import y_test as yc_test


def train_qsvm_with_credit(quantum_circuit):
    
    
    pca = PCA(n_components=len(quantum_circuit.parameters))
    Xpca_train = pca.fit_transform(Xw_train)
    Xpca_test = pca.transform(Xw_test)
    print(quantum_circuit.draw())
    print(f"Parameter: {len(quantum_circuit.parameters)}")
    backend = Aer.get_backend('statevector_simulator')
    quantum_kernel = QuantumKernel(feature_map=quantum_circuit, quantum_instance=backend)
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    qsvc.fit(Xpca_train, yc_train)
    y_pred = qsvc.predict(Xpca_test)
    accuracy = accuracy_score(yc_test, y_pred)

    return accuracy

Kiểu dữ liệu của X_train: <class 'numpy.ndarray'>, kích thước: (70, 5)
Kiểu dữ liệu của y_train: <class 'numpy.ndarray'>, kích thước: (70,)


In [9]:
import qoopmaster.evolution.generator
print(dir(qoopmaster.evolution.generator))

['Metadata', 'MetadataSynthesis', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'by_depth', 'by_depth_new', 'by_num_cnot', 'by_num_cnot_old', 'by_num_qubits', 'choice_from_array', 'constant', 'generate_random_array', 'initialize_random_parameters', 'np', 'qiskit', 'random', 'weighted_choice']


**GA**

In [10]:
from qoopmaster.evolution import generator
from qoopmaster.evolution import crossover
from qoopmaster.evolution.generator import by_num_cnot_old # By num cnot old

In [11]:
from qoopmaster.evolution import divider
from qoopmaster.evolution import normalizer
from qoopmaster.evolution import mutate
from qoopmaster.evolution import threshold
from qoopmaster.backend import constant
from qoopmaster.evolution.environment import EEnvironment


In [22]:
env_metadata = MetadataSynthesis(
    num_qubits= 4,
    num_cnot=4, 
    depth=2,  
    num_circuit=4, 
    num_generation=2,
    prob_mutate=0.01  
)


env = EEnvironment(
    metadata = env_metadata,
    fitness_func = train_qsvm_with_wine,
    generator_func = generator.by_depth_new,
    crossover_func = crossover.onepoint(
        divider.by_num_rotation_gate(int(env_metadata.num_qubits/2)),
        normalizer.by_num_rotation_gate(env_metadata.num_qubits/2)),
    mutate_func = mutate.bitflip_mutate(constant.operations),
    threshold_func = threshold.synthesis_threshold
).evol(verbose=True, mode="nonparallel")


test = env.best_circuit
print(len(test.parameters))

Step:   0%|          | 0/2 [00:00<?, ?it/s]

Initialize list of circuit ...


Step:  50%|█████     | 1/2 [00:01<00:01,  1.11s/it]

Start evol progress ...
Running at generation 1
parameter:  4
parameter:  4
parameter:  4
parameter:  4


Step: 100%|██████████| 2/2 [00:03<00:00,  1.62s/it]

[0.9231 0.8846 0.5    0.5769]
Saving circuit ...
running normalizer...
running normalizer...
Saving circuit ...
Running at generation 2
parameter:  4
parameter:  4
parameter:  5


ValueError: x_vec and class feature map have incompatible dimensions.
x_vec has 4 dimensions, but feature map has 5.

In [23]:
for i in range(0,20):
    env_metadata = MetadataSynthesis(
    num_qubits= 4,
    num_cnot=4, 
    depth=2,  
    num_circuit=4, 
    num_generation=1,
    prob_mutate=0.01  
)


    env = EEnvironment(
        metadata = env_metadata,
        fitness_func = train_qsvm_with_wine,
        generator_func = generator.by_depth_new,
        crossover_func = crossover.onepoint(
            divider.by_num_rotation_gate(int(env_metadata.num_qubits/2)),
            normalizer.by_num_rotation_gate(env_metadata.num_qubits/2)),
        mutate_func = mutate.bitflip_mutate(constant.operations),
        threshold_func = threshold.synthesis_threshold
    ).evol(verbose=True, mode="nonparallel")


    test = env.best_circuit
    print(len(test.parameters))

Initialize list of circuit ...


Start evol progress ...
Running at generation 1
parameter:  4
parameter:  4
parameter:  4
parameter:  4


Step: 100%|██████████| 1/1 [00:04<00:00,  4.80s/it]


[0.9231 0.5    0.8846 0.8077]
Saving circuit ...
running normalizer...
running normalizer...
Saving circuit ...
End evol progress, best score ever: 0.9230769230769231
5


Initialize list of circuit ...


...

[0.9231 0.8846 0.8462 0.8846]
Saving circuit ...
running normalizer...
running normalizer...
Saving circuit ...
End evol progress, best score ever: 0.9230769230769231
5


Initialize list of circuit ...


Start evol progress ...
Running at generation 1
parameter:  4
parameter:  4
parameter:  4
parameter:  4


Step: 100%|██████████| 1/1 [00:03<00:00,  3.70s/it]

[0.5    0.9231 0.9231 0.8846]
Saving circuit ...
running normalizer...
running normalizer...
Saving circuit ...
End evol progress, best score ever: 0.9230769230769231
4


In [14]:


def train_qsvm_with_Credit(quantum_circuit):
    
    # PCA - 
    pca = PCA(n_components=len(quantum_circuit.parameters))
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.fit_transform(X_test)

    print("Param: ", len(quantum_circuit.parameters))
    print("Feature: ",X_train_pca.shape[1])
    
    print(quantum_circuit.draw())
    backend = Aer.get_backend('statevector_simulator')
    quantum_kernel = QuantumKernel(feature_map=quantum_circuit, quantum_instance=backend)
    qsvc = QSVC(quantum_kernel=quantum_kernel)
    qsvc.fit(X_train_pca, y_train)
    y_pred = qsvc.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [15]:
env_metadata_credit = MetadataSynthesis(
    num_qubits= 3,
    num_cnot=3, 
    depth=3,  
    num_circuit=5, 
    num_generation=1,
    prob_mutate=0.01  
)
env = EEnvironment(
    metadata = env_metadata_credit,
    fitness_func = train_qsvm_with_Credit,
    generator_func = generator.by_depth_new,
    crossover_func = crossover.onepoint(
        divider.by_depth(int(env_metadata_credit.num_cnot/2)),
        normalizer.by_depth(env_metadata_credit.num_cnot)),
    mutate_func = mutate.bitflip_mutate(constant.operations),
    threshold_func = threshold.synthesis_threshold
).evol(verbose=True, mode="nonparallel")


Step:   0%|          | 0/1 [00:00<?, ?it/s]

Initialize list of circuit ...


KeyboardInterrupt: 

In [4]:
file_path = 'C:/Desktop/GA+QSVM/3qubits_train_qsvm_with_Credit_2024-12-13/best_circuit.qpy'

with open(file_path, 'rb') as f:
    circuits = qpy.load(f)  

test_circuit = circuits[0]

print(test_circuit)
print(f'Parameter: {len(test_circuit.parameters)} Num_qubit: {test_circuit.num_qubits}')

          ┌───┐           ┌───┐                           
q_0: ─────┤ H ├───────────┤ X ├─────────────■─────────────
     ┌────┴───┴─────┐     └─┬─┘             │        ┌───┐
q_1: ┤ Rx(theta[0]) ├───────■───────────────┼────────┤ H ├
     └──────┬───────┘┌──────────────┐┌──────┴───────┐└───┘
q_2: ───────■────────┤ Ry(theta[1]) ├┤ Rz(theta[2]) ├─────
                     └──────────────┘└──────────────┘     
Parameter: 3 Num_qubit: 3


In [22]:
train_qsvm_with_Credit(test_circuit)

          ┌───┐           ┌───┐                           
q_0: ─────┤ H ├───────────┤ X ├─────────────■─────────────
     ┌────┴───┴─────┐     └─┬─┘             │        ┌───┐
q_1: ┤ Rx(theta[0]) ├───────■───────────────┼────────┤ H ├
     └──────┬───────┘┌──────────────┐┌──────┴───────┐└───┘
q_2: ───────■────────┤ Ry(theta[1]) ├┤ Rz(theta[2]) ├─────
                     └──────────────┘└──────────────┘     


0.72